<a href="https://colab.research.google.com/github/Chuc-ngan/AIThayDu/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python tqdm

In [ ]:
!pip install tensorflow==2.18.0  # Thay đổi phiên bản theo nhu cầu

# **1. Kết nối với Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ColabNotebooks/TieuLuan'

!pwd

ValueError: mount failed

# **2. Import thư viện**

In [ ]:
# ===================== 1. Xử lý dữ liệu =====================
import os
import shutil  # Quản lý tệp/thư mục
import random
import math
import collections
import numpy as np  # Xử lý số học
import pandas as pd  # Xử lý dữ liệu CSV
from pathlib import Path
from PIL import Image, UnidentifiedImageError
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import pickle
# ===================== 2. Machine Learning =====================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import seaborn as sns

# ===================== 3. TensorFlow & Keras =====================
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import mixed_precision
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

# ===================== 4. Xây dựng mô hình =====================
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization,
    GlobalAveragePooling2D, Dense, Dropout, RepeatVector, Reshape, Flatten
)
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
from keras.layers import add, concatenate
from keras import layers, backend as K

# ===================== 5. Xử lý hình ảnh & Vẽ biểu đồ =====================
import cv2 as cv
import matplotlib.pyplot as plt

# ===================== 6. Đặt Seed (Reproducibility) =====================
tf.random.set_seed(42)  # Thay số 42 bằng giá trị seed mong muốn
np.random.seed(1)


# **3. Cấu hình môi trường TensorFlow**

In [ ]:
# Thiết lập GPU sử dụng bộ nhớ động (tránh lỗi thiếu bộ nhớ)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

# Kiểm tra TensorFlow đang chạy trên GPU hay không
print("TensorFlow is running on GPU:", tf.test.is_built_with_cuda())
# Bật chế độ Mixed Precision để tăng tốc (nếu GPU hỗ trợ)
# Thiết lập mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Bật XLA với cấu hình tối ưu
tf.config.optimizer.set_jit(True)
tf.config.optimizer.set_experimental_options({
    "layout_optimizer": True,
    "constant_folding": True,
    "shape_optimization": True,
    "remapping": True,
    "arithmetic_optimization": True,
    "dependency_optimization": True,
    "loop_optimization": True,
    "function_optimization": True,
    "debug_stripper": True
})

TensorFlow is running on GPU: True


# **4. Configuration**

In [ ]:
# Đường dẫn thư mục gốc
data_dir = "dataset/images"

# Thư mục đích
train_dir = "dataset/train"
val_dir = "dataset/validation"
test_dir = "dataset/test"

# Thư mục thiếu sáng
low_light_dir = "dataset/low_light"

low_light_noise_dir = "dataset/low_light_noise"

# Thư mục chứa ảnh sau khi tăng cường
enhanced_image_low_light_dir = "dataset/enhanced_cnn_low_light"
enhanced_image_low_light_noise_dir = "dataset/enhanced_cnn_low_light_noise"

SPLIT_RATIO = 0.5  # Chỉ lấy 50%
# Configuration
IMG_SIZE = 224  # Slightly larger for more detail
CHANEL = 3
BATCH_SIZE = 32
EPOCHS = 3
VAL_SPLIT = 0.2
RANDOM_SEED = 42

# **5. Khởi tạo mô hình InstantiateModel**
Xây dựng mạng CNN:
- Nhiều nhánh (multi-branch CNN) giúp học được nhiều đặc trưng ảnh.
- Sử dụng phép cộng (add()) để kết hợp thông tin từ các nhánh.
- Giảm dần số lượng kênh từ đầu vào → trích xuất đặc trưng → tái tạo ảnh.

In [ ]:
def get_matched_image_pairs(normal_dir, low_light_dir):
    def load_image_paths(directory):
        extensions = {".jpg", ".jpeg", ".png", ".bmp"}
        return [os.path.join(root, f)
                for root, _, files in os.walk(directory)
                for f in files
                if os.path.splitext(f.lower())[1] in extensions]

    # Load all image paths
    normal_images = load_image_paths(normal_dir)
    low_light_images = load_image_paths(low_light_dir)

    # Create filename to path mapping (without extension)
    normal_map = {os.path.splitext(os.path.basename(p))[0]: p for p in normal_images}
    low_light_map = {os.path.splitext(os.path.basename(p))[0]: p for p in low_light_images}

    # Find common filenames
    common_keys = set(normal_map.keys()) & set(low_light_map.keys())
    print(f"Found {len(common_keys)} matched image pairs")

    # Verify images actually match by checking dimensions
    verified_pairs = []
    for key in tqdm(common_keys, desc="Verifying image pairs"):
        try:
            # Quick check without loading full image
            normal_shape = cv.imread(normal_map[key], cv.IMREAD_UNCHANGED).shape
            low_shape = cv.imread(low_light_map[key], cv.IMREAD_UNCHANGED).shape

            if len(normal_shape) == 3 and len(low_shape) == 3:  # Both are color images
                verified_pairs.append((normal_map[key], low_light_map[key]))
        except:
            continue

    print(f"{len(verified_pairs)} pairs passed verification")
    return list(zip(*verified_pairs))  # Returns (normal_images, low_light_images)

# Load and verify image pairs
normal_images, low_light_images = get_matched_image_pairs(train_dir, low_light_dir)

In [ ]:
def save_verified_pairs(pairs, save_path="verified_pairs.pkl"):
    with open(save_path, "wb") as f:
        pickle.dump(pairs, f)
    print(f"Saved verified pairs to {save_path}")
save_verified_pairs((normal_images, low_light_images))

In [ ]:
def load_verified_pairs(path="verified_pairs.pkl"):
    with open(path, "rb") as f:
        normal_images, low_light_images = pickle.load(f)
    print(f"Loaded {len(normal_images)} verified pairs")
    return normal_images, low_light_images

normal_images, low_light_images = load_verified_pairs()

In [ ]:
def display_sample_pair(normal_path, low_path):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(cv.cvtColor(cv.imread(low_path), cv.COLOR_BGR2RGB))
    plt.title("Low Light")

    plt.subplot(1, 2, 2)
    plt.imshow(cv.cvtColor(cv.imread(normal_path), cv.COLOR_BGR2RGB))
    plt.title("Normal")

    plt.show()

# Hiển thị 5 cặp ngẫu nhiên
for i in np.random.choice(len(normal_images), 10):
    display_sample_pair(normal_images[i], low_light_images[i])

In [ ]:
# Train-validation split
train_norm, val_norm, train_low, val_low = train_test_split(
    normal_images, low_light_images,
    test_size=VAL_SPLIT,
    random_state=RANDOM_SEED
)

print(f"Training samples: {len(train_norm)}, Validation samples: {len(val_norm)}")

class DataGenerator:
    def __init__(self, normal_images, low_light_images, batch_size, img_size):
        self.normal = normal_images
        self.low = low_light_images
        self.batch_size = batch_size
        self.img_size = img_size
        self.indices = np.arange(len(normal_images))

    def __len__(self):
        return len(self.normal) // self.batch_size

    def __call__(self):
        while True:
            np.random.shuffle(self.indices)
            for i in range(0, len(self.indices), self.batch_size):
                batch_indices = self.indices[i:i+self.batch_size]
                batch_normal = []
                batch_low = []

                for idx in batch_indices:
                    # Load images
                    norm_img = cv.imread(self.normal[idx])
                    low_img = cv.imread(self.low[idx])

                    # Convert to RGB and resize
                    norm_img = cv.cvtColor(norm_img, cv.COLOR_BGR2RGB)
                    low_img = cv.cvtColor(low_img, cv.COLOR_BGR2RGB)

                    norm_img = cv.resize(norm_img, self.img_size)
                    low_img = cv.resize(low_img, self.img_size)

                    # Normalize to [0,1]
                    norm_img = norm_img.astype(np.float32) / 255.0
                    low_img = low_img.astype(np.float32) / 255.0

                    # Simple data augmentation - random flip
                    if np.random.rand() > 0.5:
                        norm_img = np.fliplr(norm_img)
                        low_img = np.fliplr(low_img)

                    batch_normal.append(norm_img)
                    batch_low.append(low_img)

                yield np.array(batch_low), np.array(batch_normal)

# Create data generators
train_gen = DataGenerator(train_norm, train_low, BATCH_SIZE, (IMG_SIZE, IMG_SIZE))()
val_gen = DataGenerator(val_norm, val_low, BATCH_SIZE, (IMG_SIZE, IMG_SIZE))()

In [ ]:
x, y = next(train_gen)
print(x.shape, y.shape)  # Kiểm tra batch đầu tiên có load đúng không

In [ ]:
def InstantiateModel(in_):

    model_1 = Conv2D(16,(3,3), activation='relu',padding='same',strides=1)(in_)
    model_1 = Conv2D(32,(3,3), activation='relu',padding='same',strides=1)(model_1)
    model_1 = Conv2D(64,(2,2), activation='relu',padding='same',strides=1)(model_1)

    model_2 = Conv2D(32,(3,3), activation='relu',padding='same',strides=1)(in_)
    model_2 = Conv2D(64,(2,2), activation='relu',padding='same',strides=1)(model_2)

    model_2_0 = Conv2D(64,(2,2), activation='relu',padding='same',strides=1)(model_2)

    model_add = add([model_1,model_2,model_2_0])

    model_3 = Conv2D(64,(3,3), activation='relu',padding='same',strides=1)(model_add)
    model_3 = Conv2D(32,(3,3), activation='relu',padding='same',strides=1)(model_3)
    model_3 = Conv2D(16,(2,2), activation='relu',padding='same',strides=1)(model_3)

    model_3_1 = Conv2D(32,(3,3), activation='relu',padding='same',strides=1)(model_add)
    model_3_1 = Conv2D(16,(2,2), activation='relu',padding='same',strides=1)(model_3_1)

    model_3_2 = Conv2D(16,(2,2), activation='relu',padding='same',strides=1)(model_add)

    model_add_2 = add([model_3_1,model_3_2,model_3])

    model_4 = Conv2D(16,(3,3), activation='relu',padding='same',strides=1)(model_add_2)
    model_4_1 = Conv2D(16,(3,3), activation='relu',padding='same',strides=1)(model_add)
    #Extension
    model_add_3 = add([model_4_1,model_add_2,model_4])

    model_5 = Conv2D(16,(3,3), activation='relu',padding='same',strides=1)(model_add_3)
    model_5 = Conv2D(16,(2,2), activation='relu',padding='same',strides=1)(model_add_3)

    model_5 = Conv2D(3,(3,3), activation='relu',padding='same',strides=1)(model_5)

    return model_5

In [ ]:
Input_Sample = Input(shape=(IMG_SIZE, IMG_SIZE, CHANEL)) # Đầu vào
Output_ = InstantiateModel(Input_Sample) # Kết quả đầu ra
Model_Enhancer = Model(inputs=Input_Sample, outputs=Output_)

In [ ]:
Model_Enhancer.compile(optimizer="adam", loss='mean_squared_error')
Model_Enhancer.summary()

In [ ]:
plot_model(Model_Enhancer, to_file='model_simple.png', show_shapes=True, show_layer_names=True, rankdir='LR')  # Chuyển hướng sơ đồ từ trên xuống dưới
from IPython.display import Image
Image(retina=True, filename='model_simple.png')

In [ ]:
steps_per_epoch_10percent = int(0.2 * len(train_norm) // BATCH_SIZE)
val_steps_10percent = int(0.2 * len(val_norm) // BATCH_SIZE)

In [ ]:
checkpoint = ModelCheckpoint(
    'model/model_enhance_cnn.weights.h5',  # Đúng định dạng
    monitor='val_loss',  # Theo dõi validation loss
    save_best_only=True,  # Chỉ lưu khi tốt hơn mô hình trước
    save_weights_only=True,  # Chỉ lưu trọng số
    mode='min',  # Giảm loss thì tốt hơn
    verbose=1
)

# Fit model
Model_Enhancer.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch_10percent,
    validation_steps=val_steps_10percent,
    epochs=5,  # có thể giảm xuống 3 để thử
    verbose=1,
    shuffle=True,
    callbacks=[checkpoint]
)

In [ ]:
# Load lại mô hình đã lưu
model_path = "model/model_enhance_cnn.weights.h5"  # Đường dẫn file weights
model = Model_Enhancer  # Sử dụng kiến trúc đã có
model.load_weights(model_path)

print("Mô hình đã được load thành công!")

In [ ]:
def enhance_images(input_dir, output_dir, model, img_size=(224, 224)):
    all_images = []

    # Duyệt qua toàn bộ cấu trúc thư mục
    for root, dirs, files in os.walk(input_dir):
        relative_path = os.path.relpath(root, input_dir)
        target_dir = os.path.join(output_dir, relative_path)
        os.makedirs(target_dir, exist_ok=True)

        # Lọc các tệp ảnh
        image_paths = [os.path.join(root, f) for f in files if f.endswith((".jpg", ".png"))]
        all_images.extend([(img_path, target_dir) for img_path in image_paths])

    # Thanh tiến trình duy nhất
    with tqdm(total=len(all_images), desc="Enhancing images", dynamic_ncols=True, leave=True) as pbar:
    for img_path, target_dir in all_images:
        img = cv.imread(img_path)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img = cv.resize(img, img_size)

        img = img.astype(np.float32) / 255.0
        img = np.expand_dims(img, axis=0)

        # Dự đoán không in log
        enhanced_img = model.predict(img, verbose=0)[0]
        enhanced_img = (enhanced_img * 255).astype(np.uint8)

        output_path = os.path.join(target_dir, os.path.basename(img_path))
        cv.imwrite(output_path, cv.cvtColor(enhanced_img, cv.COLOR_RGB2BGR))

        pbar.update(1)

    print(f"✅ Đã lưu {len(all_images)} ảnh cải thiện vào {output_dir}")

# Gọi hàm tăng cường ảnh
enhance_images(low_light_test_dir, enhanced_image_dir, Model_Enhancer)